In [33]:
from matminer.data_retrieval.retrieve_MP import MPDataRetrieval
from matminer.datasets import load_dataset
from pymatgen.core.structure import Structure
import pandas as pd
import requests
import shutil
from matminer.featurizers.structure import OrbitalFieldMatrix
import numpy as np
import pickle as pkl

In [29]:

T = ['Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au']
LA = ['La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu']
X = ['B', 'C', 'N', 'O']

print(len(T))
print(len(LA))
print(len(X))

criteria = []

for i in T:
    m = ''
    m = i
    for j in T:
        criteria.append(m+j)
        
for i in LA:
    m = ''
    m = i
    for j in T:
        criteria.append(m+j)

for i in LA:
    m = ''
    m = i
    for j in T:
        m1 = m
        m1 += j
        for k in X:
            criteria.append(m1+k)
            

len(criteria)


In [30]:
filename = './mp-ids-46744.txt'
with open(filename) as f:
    criteria = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
criteria = [x.strip() for x in criteria] 
criteria = criteria[:-1]

In [31]:
print('Number of materials: {}'.format(len(criteria)))

Number of materials: 46744


In [1]:
def function(criteria):
    mpr = MPDataRetrieval(api_key="I7dlHeJv9K0gnqC2")

    properties = ['pretty_formula', 'structure', 'nsites', 'formation_energy_per_atom']

    df = pd.DataFrame()
    for m_id in criteria:
        df1 = mpr.get_dataframe(criteria=m_id, properties=properties)
        df = pd.concat([df, df1])

    df['formation_energy'] = df['formation_energy_per_atom'] * df['nsites']
    df['structure'] = pd.Series([Structure.from_dict(df['structure'][i])\
        for i in range(df.shape[0])], df.index)

    df.to_pickle('./cgcnn_df')
    df

In [20]:
def download_cif(df, foldername):
    print("downloading cif files...............")
    index = df.index
    for it, idx in enumerate(index):
        url = 'https://www.materialsproject.org/materials/'+idx+'/cif?type=computed'
        filename = './{}/cgcnn/{}_{}.cif'.format(foldername, it, idx)

        r = requests.get(url, stream=True)
        if r.status_code == 200:
            with open(filename, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)

In [21]:
def get_ofm_matrix(df):
    ofm = OrbitalFieldMatrix(period_tag=False)
    # ofm.set_n_jobs(10)
    print("calculating ofm...............")
    df = ofm.fit_featurize_dataframe(df, 'structure')
    print("done")
    return df

In [22]:
def download_ofm_matrix(df, filename, foldername):
    print("downloading ofm matrices...............")
    keys = np.array(df.keys())
    keys = keys[5:]

    ndf = df[keys].values
    num_crystal = int(ndf.size/ndf[0].size)
    ndf = np.reshape(ndf, (num_crystal, 32, 32))

    labels = df['formation_energy'].values
    print(ndf.shape)
    print(labels.shape)

    pkl.dump(ndf, open("./{}/ofm/{}_x.pkl".format(foldername, filename), "wb"))
    pkl.dump(labels, open("./{}/ofm/{}_y.pkl".format(foldername, filename), "wb"))

In [23]:
# mpr = MPDataRetrieval(api_key="I7dlHeJv9K0gnqC2")

# properties = ['pretty_formula', 'structure', 'nsites', 'formation_energy_per_atom']

# df1 = mpr.get_dataframe(criteria="*-O", properties=properties)

# df1['formation_energy'] = df1['formation_energy_per_atom'] * df1['nsites']
# df1['structure'] = pd.Series([Structure.from_dict(df1['structure'][i])\
#     for i in range(df1.shape[0])], df1.index)

# df1

In [24]:
# df1.to_pickle('./oxides_df')

In [25]:
# oxides_df = pd.read_pickle('./oxides_df')
# oxides_df = get_ofm_matrix(oxides_df)
# download_ofm_matrix(oxides_df, filename='ofm_dataset', foldername='dataset_oxides')
# download_cif(oxides_df, foldername='dataset_oxides')

In [26]:
cgcnn_df = pd.read_pickle('./cgcnn_df')
cgcnn_df = get_ofm_matrix(cgcnn_df)
cgcnn_df
download_ofm_matrix(cgcnn_df, filename='ofm_dataset', foldername='dataset_cgcnn')
download_cif(cgcnn_df, foldername='dataset_cgcnn')

calculating ofm...............



done
downloading ofm matrices...............
(1, 32, 32)
(1,)
downloading cif files...............
